# Imports

In [15]:
from glob import glob

import pandas as pd
import numpy as np

## Data Loading

In [ ]:
!git clone https://github.com/usnistgov/trec_eval.git
!cd trec_eval && make

In [17]:
#folder where retrieval results are stored, file name should be {alg}.trec
l1_retrievers_trec_folder = "regnlp-test-l1"

In [18]:
results = []
for e in glob(f"/kaggle/input/{l1_retrievers_trec_folder}/*.trec"):
    retrieval_results = pd.read_csv(e, sep=" ", names=["qid", "_", 'pid', 'ind', 'score', "alg"])
    retrieval_results['alg'] = e.split("/")[-1].split(".")[0]
    results.append(retrieval_results)
df = pd.concat(results).reset_index(drop=True).drop_duplicates(subset=["qid", "pid", "alg"])
retrieval_algorithms = df['alg'].unique()
df = df.set_index(['qid', 'alg']).sort_index()

## Algorithm

In [19]:
# naive implementation of RRF 
rrf_q = 4
questions = np.unique([e[0] for e in df.index])
final_answers = []
for each_question in (questions):
    passages = {}
    for each_alg in retrieval_algorithms:
        subset = df.loc[(each_question, each_alg)].sort_values(by=['score'], ascending=False)
        count = 0
        for each_passage in list(subset['pid']):
            if each_passage not in passages:
                passages[each_passage] = 0
            passages[each_passage]+=(1/(rrf_q+count))
            count += 1
    count = 0
    for each_passage in sorted(passages, key=lambda e: -1*passages[e])[:50]:
        final_answers.append([each_question, "0",  each_passage, count, passages[each_passage], 'alg'])
        count += 1
pd.DataFrame(final_answers).drop_duplicates(subset=[0, 2]).to_csv("./rrf.trec", index=False, sep=" ", header=False)

In [ ]:
!trec_eval/trec_eval -m recall.10 -m map_cut.10 gt.qrels ./rrf.trec